In [ ]:
##module to get timeseries of the regional average of a variable

In [1]:
import modules as md 
import xarray as xr
xr.set_options(display_style='html')
import intake
import cftime
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import s3fs
%matplotlib inline

In [2]:

def regional_average(model,min_lat,max_lat,min_lon,max_lon,var):

    cell_area=md.get_areacello(model,min_lat,max_lat,min_lon,max_lon)

    s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", 
                       secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0", client_kwargs=dict(endpoint_url="https://rgw.met.no"))

    files_dir='/escience2022/Ada/monthly/tos_Omon_NorESM2-LM_omip1_r1i1p1f1_gn_20*.nc'

    remote_files = 's3:/'+ files_dir
    remote_files = s3.glob(remote_files)
    fileset = [s3.open(file) for file in remote_files]
    ds = xr.open_mfdataset(fileset, combine='by_coords')
    
    month_length = ds.time.dt.days_in_month
    weights = month_length.groupby("time.year") / month_length.groupby("time.year").sum()
    # Test that the sum of the weights for each year is 1.0
    np.testing.assert_allclose(weights.groupby("time.year").sum().values, np.ones(len(np.unique(ds.time.dt.year))))
    # Calculate the weighted average:
    da = (ds.var * weights).groupby("time.year").sum(dim="time")
    da = da.isel(year = slice(10,None))
    
    BSsst = da.where((da.latitude>=min_lat) & (da.latitude<=max_lat) & (da.longitude <= min_lon)  & (da.longitude >= max_lon))
    return BSsst


min_lat= 60
max_lat= 90
min_lon= 0
max_lon= 360
var='tos'
model= 'NorESM2-LM'

BSsst=regional_average(model,min_lat,max_lat,min_lon,max_lon,var)
BSsst.mean(dim='year').plot.contourf()
    



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


TypeError: unsupported operand type(s) for *: 'method' and 'float'